# Scaling document clustering and data deduplication with SageMaker HuggingFace and K-Means 

## Installing and importing packages

In [1]:
#%pip install -U sentence-transformers

In [2]:
import pandas as pd
import numpy as np
import torch
import boto3
import sagemaker
from sentence_transformers import SentenceTransformer
from sagemaker.deserializers import JSONDeserializer

## Data prepration 
The dataset used is a Quora dataset that has a pair of sentence with a label that classifies if the sentences are similar or different. In our example we are only in the sentences themself, so we will create a single list that includes the sentences from both columns

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.shape

(404290, 6)

In [5]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [6]:
# Create a single list of sentences 
sentences = df['question1'].to_list()
sentences2 = df['question2'].to_list()
sentences.extend(sentences2)

In [7]:
len(sentences) 

808580

In [9]:
sentences[0:5]

['What is the step by step guide to invest in share market in india?',
 'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
 'How can I increase the speed of my internet connection while using a VPN?',
 'Why am I mentally very lonely? How can I solve it?',
 'Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?']

## Create embeddings
In this section we will a pertained sentence transformer to create embeddings for our sentences. We will use these embeddings to create our final cluster 

In [10]:
# initialize sentence transformer model
model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [15]:
%%time
sentence_embeddings = model.encode(sentences) #create embeddings

CPU times: user 30min 55s, sys: 3min 58s, total: 34min 53s
Wall time: 5min 9s


In [16]:
sentence_embeddings.shape #check embeddings

(808580, 768)

## SageMaker Training
In this section we will transform the embeddings into recordIO format, shard them across S3 key and train a k-means classifier on SageMaker

### Import packages and setup SageMaker

In [19]:
import sagemaker
import boto3
from sagemaker import KMeans 

sess = sagemaker.Session() #session
role = sagemaker.get_execution_role() # get role
region = boto3.Session().region_name #get default region
s3_client = boto3.client("s3") 
bucket = sess.default_bucket() # get default bucket
prefix = "sagemaker/data-dedup"

print(role)
print(bucket)

arn:aws:iam::967669495843:role/service-role/AmazonSageMaker-ExecutionRole-20190812T143756
sagemaker-us-east-1-967669495843


### Hyperparameters and Data
Setup up the number of clusters, training data location and the output path. The training data location is where your transformed data that is sharded by S3 keys is uploaded and the output path will be used to write your model artifacts after then end of training.

In [21]:
num_clusters = 15000 # number of clusters
key = 'train-data'
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
output_path="s3://sagemaker-us-east-1-967669495843/sagemaker/data-dedup/output"
print(f"Training Data Loaction: %s"% s3_train_data)
print(f"Output Data Loaction: %s"% output_path)

Training Data Loaction: s3://sagemaker-us-east-1-967669495843/sagemaker/data-dedup/train/train-data
Output Data Loaction: s3://sagemaker-us-east-1-967669495843/sagemaker/data-dedup/output


### Create Estimator and start training
Here we create our SM estimator, specify the input and output locations and being training.We use the record_set method to convert the embeddings into recordIO and shard them by S3 key prefixes and upload it to the training data location. 

In [22]:
kmeans = KMeans(
    role=role,
    instance_count=2,
    instance_type="ml.c4.xlarge",
    output_path=output_path,
    k=num_clusters,
    data_location=s3_train_data
)

In [23]:
kmeans.fit(kmeans.record_set(sentence_embeddings)) # start training

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2022-03-08 22:40:28 Starting - Starting the training job...
2022-03-08 22:40:56 Starting - Preparing the instances for trainingProfilerReport-1646779228: InProgress
............
2022-03-08 22:42:53 Downloading - Downloading input data......
2022-03-08 22:43:57 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
Docker entrypoint called with argument(s): train
Running default environment configuration script
[03/08/2022 22:44:04 INFO 139672847439680 integration.py:636] worker started
[03/08/2022 22:44:04 INFO 139672847439680] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'init_method': 'random', 'mini_batch_size': '5000', 'epochs': '1', 'extra_center_factor': 'auto', 'local_lloyd_max_iter': '300', 'local_lloyd_tol': '0.0001', 'local_lloyd_init_method': 'kmeans++', 'local_lloyd_num_trials': 'auto', 'half_li

## Inference
In this section we will create a SageMaker endpoint and use it to get inferences. In our case this will be a cluster id for each of the sentence embeddings in our dataset. Since we are using real time endpoints to run inference, we will split our embeddings into multiple batches of smaller size and pass these batches to the endpoint for inference.

In [26]:
# create batches
import numpy as np
batches = np.array_split(sentence_embeddings, 10000)
print(f"data split into 10000 batches, of size {batches[0].shape[0]}.")

data split into 10000 batches, of size 81.


Create endpoint

In [27]:
%%time
kmeans_predictor = kmeans.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-------!CPU times: user 226 ms, sys: 9.17 ms, total: 235 ms
Wall time: 3min 31s


We use the `JSONDeserializer()` method from `sagemaker.deserializers` which makes it easy to work with inference results. Here we collate the results from all the batches.

In [29]:
from sagemaker.deserializers import JSONDeserializer
kmeans_predictor.deserializer = JSONDeserializer() 

In [30]:
%%time
batch_results =[]
for batch in batches:
    result = kmeans_predictor.predict(batch)
    result=result["predictions"]
    for r in range(len(result)):
        batch_results.append(result[r]['closest_cluster'])

CPU times: user 2min 23s, sys: 5.43 s, total: 2min 28s
Wall time: 22min 56s


In [31]:
len(batch_results) #verify inference results

808580

## Evaluation

### Post Processing
Our inference results consist of cluster id for each sentence embedding in our dataset. Here we take the results from our inference and map them to our original sentences. We also convert everything into a data frame that makes data analysis easier.

In [32]:
sentence_clusters = list(map(list,zip(sentences,batch_results)))

In [33]:
# convert to data frame
sentence_clusters_df = pd.DataFrame(sentence_clusters,columns=['text','cluster_id'])
sentence_clusters_df.head(10)

,text,cluster_id
0,What is the step by step guide to invest in sh...,4448.0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,4780.0
2,How can I increase the speed of my internet co...,3974.0
3,Why am I mentally very lonely? How can I solve...,1831.0
4,"Which one dissolve in water quikly sugar, salt...",1338.0
5,Astrology: I am a Capricorn Sun Cap moon and c...,1156.0
6,Should I buy tiago?,12979.0
7,How can I be a good geologist?,332.0
8,When do you use シ instead of し?,1314.0
9,Motorola (company): Can I hack my Charter Moto...,219.0


### Analyzing results

Let's check some random clusters, like here check a cluster related to investing shows good results.

In [34]:
sentence_clusters_df[sentence_clusters_df["cluster_id"] == 4448.0]

,text,cluster_id
0,What is the step by step guide to invest in sh...,4448.0
11206,How do I invest money into stock market?,4448.0
11387,What all does somebody need to know to start i...,4448.0
13805,How do I invest in a share market?,4448.0
14829,How do I invest in stock market?,4448.0
...,...,...
789808,What is the procedure to invest in the share m...,4448.0
793433,How did Rakesh Jhunjhunwala make money to inve...,4448.0
798449,I am software enginner and I want to invest in...,4448.0
801246,How do I invest profitably in stock market?,4448.0


Similarly if we pick up another cluster presumably about internet connectivity we see similar results.

In [40]:
sentence_clusters_df[sentence_clusters_df["cluster_id"] == 3974.0]

,text,cluster_id
2,How can I increase the speed of my internet co...,3974.0
1918,How can I speed up my Internet connectionn?,3974.0
5420,I have a Tata Photon Plus dongle for the inter...,3974.0
21934,How can I speed up my Internet connection?,3974.0
44264,My upload speed in uTorrent is triple or even ...,3974.0
...,...,...
788986,How can I boost up Internet speed or hack inte...,3974.0
789231,What causes Comcast internet to be slow?,3974.0
791298,My JIOFI device is not giving speed even more ...,3974.0
793570,I am paying for 20Mbps internet but getting on...,3974.0


Even in cases for very niche subjects like  documents about the Kohinoor diamond, yields some decent results

In [39]:
sentence_clusters_df[sentence_clusters_df["cluster_id"] == 4780.0]

,text,cluster_id
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,4780.0
17296,What would happen if the Indian government sto...,4780.0
81998,Do you think Kohinoor was a gift from India?,4780.0
131477,Will India ever get its Kohinoor diamond back?,4780.0
133496,What if Kohinoor diamond comes back to India?,4780.0
178151,Why England is not returning Kohinoor Diamond ...,4780.0
184732,What is the story of Kohinoor (Koh-i-Noor) Dia...,4780.0
241715,Is it possible to get Kohinoor from British ro...,4780.0
263614,What is the story of Kohinoor (Koh-i-Noor) Dia...,4780.0
313927,Was the Kohinoor really gifted to England queen?,4780.0


There are some areas where it did not perform as well. for example in cases where there were many small instances very similar documents, but the differences were still meaningful, the model ended up clustering them within the same cluster.For example, this specific cluster includes documents related to geology and other similar  and general sciences.

In [41]:
sentence_clusters_df[sentence_clusters_df["cluster_id"] == 332.0]

,text,cluster_id
7,How can I be a good geologist?,332.0
13674,What should I consider before becoming a chef?,332.0
17614,How do I become better at research?,332.0
21951,How does one become a scientist?,332.0
37569,I want to become an astronomer but how?,332.0
...,...,...
772626,"My IQ level is 105 and I am of 14 years, can I...",332.0
781888,How can I become an forensic scientist?,332.0
788732,How do I become an Opthalmologist?,332.0
800644,How can I become an astrophysicist in India? I...,332.0


## Clean-up
Optionally the endpoint created to stop incurring charges. You may also want to remove files from S3 to avoid any storage charges.

In [42]:
kmeans_predictor.delete_endpoint()

## Conclusion
In this example we showcased how to use HuggingFace embeddings along with SageMaker k-means algorithm for clustering similar documents and find any duplicates. This can be used as a viable approach other index based document similarity approaches.